In [1]:
# Q-Learning in cartpole
# interesting tutorial for this example here
#https://medium.com/swlh/using-q-learning-for-openais-cartpole-v1-4a216ef237df

In [2]:
import numpy as np
import gymnasium as gym
import session_info
import time
import math

In [3]:
# Q-Learning parameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.9  # Epsilon-greedy policy
epsilon_decay_value = 0.99995

# Sepcific Cartpole parameters
# <cart position, cart velocity, pole angle, pole velocity>

Observation = [30, 30, 50, 50]                            # weights importance of features
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])     # steps based -WHAT- PLEASE CORRECT

# Fix these
episodes = 60000  # Number of episodes

LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 60000
total = 0
total_reward = 0
prior_reward = 0

Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 1

epsilon_decay_value = 0.99995


LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 60000
total = 0
total_reward = 0
prior_reward = 0

Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 1

epsilon_decay_value = 0.99995

In [4]:
# Initialize the environment and Q table
env = gym.make('CartPole-v1', render_mode='rgb_array')

Q = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
print(Q.shape)

def get_discrete_state(state):
    discrete_state = state/np_array_win_size+ np.array([15,10,1,10])
    return tuple(discrete_state.astype(int))

state, _ = env.reset()
print(state)
print('state', state, 'discrete state', get_discrete_state(state))
print(type(get_discrete_state(state)))
print(Observation)

def choose_action(Q, discrete_state):
    if np.random.random() > epsilon:
        action = np.argmax(Q[discrete_state])               # exploration 
    else:
        action = np.random.randint(0, env.action_space.n)         # explotaition
    return action


(30, 30, 50, 50, 2)
[-0.04951638 -0.00878721 -0.02637554  0.02259252]
state [-0.04951638 -0.00878721 -0.02637554  0.02259252] discrete state (14, 9, -1, 10)
<class 'tuple'>
[30, 30, 50, 50]


In [5]:
for episode in range(episodes):
    
    t0 = time.time()  
    state, _ = env.reset()
    discrete_state = get_discrete_state(state) 
    done = False
    episode_reward = 0 
       
    while not done: 
        action = choose_action(Q, discrete_state)                    # Exploration or explotaition

        new_state, reward, done, _ , _ = env.step(action) 
        episode_reward += reward                                  # reward by episode
        new_discrete_state = get_discrete_state(new_state)

        if episode % 2000 == 0:                                   # one visualization every 2000 episodes
            env.render()

        if not done: 
            # Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])
            
            max_future_q = np.max(Q[new_discrete_state])
            current_q    = Q[discrete_state + (action,)]
            new_q        = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            Q[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    if epsilon > 0.05:                                            #epsilon modification
        if episode_reward > prior_reward and episode > 10000:
            epsilon = math.pow(epsilon_decay_value, episode - 10000)

    t1 = time.time()                                             #episode has finished
    episode_total = t1 - t0                                      #episode total time
    total = total + episode_total

    total_reward += episode_reward      #episode total reward
    prior_reward = episode_reward

    if (episode % 1000) == 0:           #every 100 episodes print the average time and the average reward
        mean = total / 1000
        total = 0
        mean_reward = total_reward / 1000
        print("Episode: " + str(episode))
        print("   Epsilon: " + str(epsilon))
        print("   Time Average: " + str(mean))
        print("   Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()

Episode: 0
   Epsilon: 1
   Time Average: 6.753301620483399e-05
   Mean Reward: 0.01
Episode: 1000
   Epsilon: 1
   Time Average: 0.0002330312728881836
   Mean Reward: 21.891
Episode: 2000
   Epsilon: 1
   Time Average: 0.00026174545288085936
   Mean Reward: 22.298
Episode: 3000
   Epsilon: 1
   Time Average: 0.00022989988327026367
   Mean Reward: 21.867
Episode: 4000
   Epsilon: 1
   Time Average: 0.0002513740062713623
   Mean Reward: 22.22
Episode: 5000
   Epsilon: 1
   Time Average: 0.0002355060577392578
   Mean Reward: 22.39
Episode: 6000
   Epsilon: 1
   Time Average: 0.0013694701194763184
   Mean Reward: 21.873
Episode: 7000
   Epsilon: 1
   Time Average: 0.0002491662502288818
   Mean Reward: 22.726
Episode: 8000
   Epsilon: 1
   Time Average: 0.00029605555534362795
   Mean Reward: 22.605
Episode: 9000
   Epsilon: 1
   Time Average: 0.0002346651554107666
   Mean Reward: 22.376
Episode: 10000
   Epsilon: 1
   Time Average: 0.0002735145092010498
   Mean Reward: 22.094
Episode: 1100

In [9]:
# Testing results from Q Table

env = gym.make('CartPole-v1', render_mode='human')
state, _ = env.reset()
k = 0
# Run the environment for 100 steps
for i in range(1000):
    # Display the current state of the environment
    env.render()
    
    # Choose an action based on the learned Q-network
    discrete_state = get_discrete_state(state)
    action = choose_action(Q, discrete_state)
    
    # Take the chosen action and observe the next state, reward, and termination status
    state, reward, terminated, truncated, _ = env.step(action)
    
    # If the episode is terminated or truncated, reset the environment
    if terminated or truncated:
        print('Terminated', i-k)
        k = i
        state, info = env.reset()

# Close the environment after exploration
env.close()

Terminated 200
Terminated 81
Terminated 117
Terminated 209
Terminated 201
Terminated 137


In [10]:
session_info.show(html=False)

-----
gymnasium           0.29.1
numpy               1.26.4
session_info        1.0.0
-----
IPython             8.26.0
jupyter_client      8.6.2
jupyter_core        5.7.2
-----
Python 3.12.3 (main, Sep 11 2024, 14:17:37) [GCC 13.2.0]
Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.39
-----
Session information updated at 2024-09-19 16:55
